**Setup the application**

In [1]:
import glob, sys, threading

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, fetch_tables, WikidataSemanticModelHelper

from sm_widgets.widgets.annotator import Annotator, BatchAnnotator
from sm_widgets_integration.wikidata import get_qnode_db, get_ontclass_db, get_ontprop_db
from sm_widgets_integration.with_grams import convert_table
from sm_widgets.widgets.slider import DisplayShell, Slider

In [2]:
cwd = ROOT_DIR / "examples/misc"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

**Fetch tables from a webpage and select table that we want to annotate**

In [3]:
tables = fetch_tables("https://en.wikipedia.org/wiki/List_of_largest_selling_pharmaceutical_products")
# tables = fetch_tables("https://en.wikipedia.org/wiki/Novartis")

In [4]:
slider = Slider(app=DisplayShell(render_fn=lambda df: display(df)), app_render_fn='set_data', embedded_error_log=True)
slider.render()

threading.Thread(
    target=lambda: slider.set_data([
        {"description": "<br>" + " > ".join([r.header for r in table.context]), "args": [table.as_df()]}
        for table in tables
    ])
).start()

HTML(value='<div id="ffbc0f8c-820c-48b0-b6d9-68f75b552d71"></div>')

Output()

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='a84666d8-62ed-4dd4-969d-4d2d63bea7d7')

Output()

<IPython.core.display.Javascript object>

**Annotate the selected table**

In [5]:
selected_table = 3

table = tables[selected_table]
table = table.as_relational_linked_table(table_id=f"{table.page_url}/{selected_table}")
annotation = grams.annotate(table)

**Show the annotation**

In [6]:
assistant = None
annotator = Annotator(
    entities=get_qnode_db(grams.qnodes), 
    ontclasses=get_ontclass_db(grams.wdclasses), 
    ontprops=get_ontprop_db(grams.wdprops), 
    prop_type_uri='http://www.wikidata.org/prop/P31', 
    savedir=cwd / "ground-truth", 
    eshost='http://mira.isi.edu:9200', 
    dev=False, 
    assistant=assistant)
annotator.render(True)

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='053f0625-c56f-4144-82e7-0f94d938e150')

HTML(value='<div id="1d930620-19d7-4f92-ab94-33d552c16659"></div>')

<IPython.core.display.Javascript object>

In [7]:
threading.Thread(
    target=lambda: annotator.set_data(table.id, convert_table(table), sms=[annotation.sm])
).start()